### Custome text entities recognition
Using Azure AI Language studio to train and deploy model, API for prediction. Based on [this](https://microsoftlearning.github.io/mslearn-ai-language/Instructions/Exercises/05-extract-custom-entities.html) Microsoft learn manual.

In [ ]:
%pip install azure-ai-textanalytics==5.3.0

In [1]:
from dotenv import load_dotenv
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

# Get Configuration Settings
load_dotenv()
ai_endpoint = os.getenv('AI_SERVICE_ENDPOINT')
ai_key = os.getenv('AI_SERVICE_KEY')
project_name = os.getenv('PROJECT')
deployment_name = os.getenv('DEPLOYMENT')
print(f'Project: {project_name} - Deployment: {deployment_name}')

Project: CustomEntityLab - Deployment: AdEntities


In [2]:
# Read each text file in the ads folder
batchedDocuments = []
ads_folder = os.path.join (os.getcwd(), "data", "ads")
files = os.listdir(ads_folder)
for file_name in files:
    # Read the file contents
    text = open(os.path.join(ads_folder, file_name), encoding='utf8').read()
    batchedDocuments.append(text)
    print(f"Added {file_name} to the batch")
print(f"Total {len(batchedDocuments)} documents added to the batch")
print(batchedDocuments)

Added test1.txt to the batch
Added test2.txt to the batch
Total 2 documents added to the batch
["Bluetooth earbuds, $100. These work okay, but sometimes disconnect from the phone. I'm sure someone more technical that me could figure it out.\n\nLocated in Sacramento, CA", 'Dog harness for sale, $20. Good condition, puppy just outgrew it.\n\nLocal meet up in Tucson, AZ in a public location.']


In [3]:
# Create client using endpoint and key
credential = AzureKeyCredential(ai_key)
ai_client = TextAnalyticsClient(endpoint=ai_endpoint, credential=credential)

In [4]:
# Extract entities
operation = ai_client.begin_recognize_custom_entities(
    batchedDocuments,
    project_name=project_name,
    deployment_name=deployment_name
)
document_results = operation.result()

In [5]:
# Display the results
for doc, custom_entities_result in zip(files, document_results):
    print(doc)
    if custom_entities_result.kind == "CustomEntityRecognition":
        for entity in custom_entities_result.entities:
            print(f"\tEntity '{entity.text}' has category '{entity.category}' with confidence score of '{entity.confidence_score}'")
    elif custom_entities_result.is_error is True:
        print("\tError with code '{custom_entities_result.error.code}' and message '{custom_entities_result.error.message}'")

test1.txt
	Entity 'Bluetooth earbuds,' has category 'ItemForSale' with confidence score of '0.9'
	Entity '$100' has category 'Price' with confidence score of '0.96'
	Entity 'Sacramento, CA' has category 'Location' with confidence score of '0.98'
test2.txt
	Entity 'Dog harness' has category 'ItemForSale' with confidence score of '0.78'
	Entity '$20' has category 'Price' with confidence score of '0.99'
	Entity 'Tucson, AZ' has category 'Location' with confidence score of '0.99'
